In [3]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import joblib
import pickle


In [4]:
dim_reduced_df = pd.read_csv('../data/feature-engineered/df_2_reduced.csv')

In [5]:
dim_reduced_df.shape

(700, 52)

In [8]:
dim_reduced_df.dtypes

date                                      object
1_year_rate                              float64
3_months_rate                            float64
6_months_rate                            float64
CPI                                      float64
INDPRO                                   float64
10_year_rate                             float64
share_price                              float64
unemployment_rate                        float64
PPI                                      float64
OECD_CLI_index                           float64
CSI_index                                float64
gdp_per_capita                           float64
seasonally_adjusted_unemployment_rate    float64
CPI_trend                                float64
CPI_sumsq_acf_diff1                      float64
seasonally_adjusted_CPI                  float64
INDPRO_trend                             float64
seasonally_adjusted_INDPRO               float64
INDPRO_acf1_original                     float64
CPI_sumsq_acf_origin

In [9]:
dim_reduced_df = dim_reduced_df.drop(['3_month_recession_probability.1', 'recession_probability.1'], axis=1)


In [10]:
dim_reduced_df.to_csv('../data/feature-engineered/df_2_reduced_final.csv', index=False)

In [11]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR

warnings.filterwarnings("ignore")

# ---- CONFIG ----
TARGET_COLS = [
    'share_price', 'gdp_per_capita'
]

RECESSION_COLS = [
    'recession_probability','1_month_recession_probability',
    '3_month_recession_probability','6_month_recession_probability'
]

SPLIT_DATE = '2020-01-01'   # test set starts here

# -------------------------
# ---- Utility / Pipeline
# -------------------------
def make_splits(df, split_date=SPLIT_DATE):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').set_index('date')

    train = df.loc[df.index < split_date].copy()
    test  = df.loc[df.index >= split_date].copy()

    print(f"Train: {train.index.min().date()} → {train.index.max().date()} ({len(train)})")
    print(f"Test : {test.index.min().date()} → {test.index.max().date()} ({len(test)})")
    return train, test


def build_exog(df, target_cols=TARGET_COLS, recession_cols=RECESSION_COLS):
    # Exogenous features = everything except targets + recession cols
    drop_cols = set(target_cols) | set(recession_cols)
    exog_cols = [c for c in df.columns if c not in drop_cols]
    return df[exog_cols].copy(), exog_cols


def clean_exog(train_exog, test_exog):
    # Fill, replace inf, align columns in same order
    def _clean(X):
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.ffill().bfill()
        return X

    train_exog = _clean(train_exog)
    test_exog  = _clean(test_exog)

    # Remove constant columns (no variance) using train only
    keep = train_exog.nunique() > 1
    train_exog = train_exog.loc[:, keep]
    test_exog  = test_exog[train_exog.columns]  # same columns/order

    # Final safety fill
    test_exog  = test_exog.fillna(method='ffill').fillna(method='bfill')
    train_exog = train_exog.fillna(method='ffill').fillna(method='bfill')
    return train_exog, test_exog


def difference_targets(train_y, test_y):
    """ First-difference targets for stationarity and keep last train level to invert later. """
    dy_train = train_y.diff().dropna()
    # For test, we will forecast differences; inversion uses last train level
    last_train_level = train_y.iloc[-1]
    return dy_train, last_train_level


def invert_differences(diff_forecast_df, last_train_level):
    """ Convert differenced forecasts back to levels via cumulative sum. """
    # cumulative sum across time, add last_train_level (broadcast on columns)
    levels = diff_forecast_df.cumsum()
    for col in levels.columns:
        levels[col] = last_train_level[col] + levels[col]
    return levels


def fit_varmax_and_forecast(train_y, test_h, train_exog=None, test_exog=None, order_tuple=(1,0)):
    """
    Fit VARMAX on (train_y) with optional exog, forecast 'test_h' steps ahead,
    return forecasts (DataFrame) and the fitted results object.

    order_tuple is (p, q)
    """
    p, q = order_tuple
    if train_exog is not None and test_exog is not None:
        model = VARMAX(endog=train_y, exog=train_exog, order=(p, q), trend='n')
        res = model.fit(maxiter=500, disp=False)
        fcast = res.get_forecast(steps=test_h, exog=test_exog).predicted_mean
    else:
        model = VARMAX(endog=train_y, order=(p, q), trend='n')
        res = model.fit(maxiter=500, disp=False)
        fcast = res.get_forecast(steps=test_h).predicted_mean

    # Build a sensible index for the forecast: start at next period after train_y
    freq = pd.infer_freq(train_y.index)
    if freq is None:
        # fallback: use train frequency as difference
        diff = train_y.index[1] - train_y.index[0]
        start = train_y.index[-1] + diff
        fcast.index = pd.date_range(start=start, periods=test_h, freq=diff)
    else:
        start = train_y.index[-1] + pd.tseries.frequencies.to_offset(freq)
        fcast.index = pd.date_range(start=start, periods=test_h, freq=freq)

    fcast.columns = train_y.columns
    return fcast, res


def fit_var_fallback(train_y, test_h, order=2):
    """
    Fallback: plain VAR on (train_y) (no exog).
    """
    var_model = VAR(train_y)
    res = var_model.fit(order)
    fcast_vals = res.forecast(train_y.values[-order:], steps=test_h)
    freq = pd.infer_freq(train_y.index)
    if freq is None:
        diff = train_y.index[1] - train_y.index[0]
        start = train_y.index[-1] + diff
        index = pd.date_range(start=start, periods=test_h, freq=diff)
    else:
        start = train_y.index[-1] + pd.tseries.frequencies.to_offset(freq)
        index = pd.date_range(start=start, periods=test_h, freq=freq)

    fcast = pd.DataFrame(fcast_vals, index=index, columns=train_y.columns)
    return fcast, res


def evaluate_forecasts(actual_df, pred_df):
    """ Return MAE/RMSE/MAPE per series. """
    common = actual_df.index.intersection(pred_df.index)
    A = actual_df.loc[common]
    P = pred_df.loc[common]

    metrics = []
    for col in A.columns:
        a = A[col].astype(float)
        p = P[col].astype(float)
        mae  = np.mean(np.abs(p - a))
        rmse = np.sqrt(np.mean((p - a)**2))
        # avoid division by zero in MAPE
        denom = np.where(a == 0, np.nan, np.abs(a))
        mape = np.nanmean(np.abs((a - p) / denom)) * 100
        metrics.append({'series': col, 'MAE': mae, 'RMSE': rmse, 'MAPE_%': mape})
    return pd.DataFrame(metrics).set_index('series')

# -------------------------
# ---- Metaheuristic helpers (lightweight implementations)
# -------------------------
def fitness_VARMAX(order_tuple, train_y, val_y, train_exog=None, val_exog=None):
    """
    Fit VARMAX(order_tuple) on train_y (and train_exog) and compute RMSE on val_y.
    Return RMSE (lower is better). If fit fails, return large penalty.
    """
    try:
        forecast, _ = fit_varmax_and_forecast(
            train_y=train_y,
            test_h=len(val_y),
            train_exog=train_exog,
            test_exog=val_exog,
            order_tuple=order_tuple
        )
        # Align indices (forecast index may differ but shapes match)
        # compute RMSE across all series averaged
        # if shape mismatch, treat as failure
        if forecast.shape != val_y.shape:
            return np.inf
        rmse = np.sqrt(((val_y.values - forecast.values) ** 2).mean())
        return float(rmse)
    except Exception:
        return np.inf


def grey_wolf_optimize_p(train_y, val_y, train_exog=None, val_exog=None,
                         p_min=1, p_max=5, pop_size=6, max_iters=10):
    """
    Simple integer Grey Wolf Optimizer to search p in [p_min, p_max].
    Returns best_p (int).
    """
    # Initialize wolves randomly (integers)
    wolves = np.random.randint(p_min, p_max+1, size=pop_size)
    fitness = np.array([fitness_VARMAX((int(w), 0), train_y, val_y, train_exog, val_exog) for w in wolves])

    # ensure we have finite values
    for it in range(max_iters):
        print(f"[GWO] Iteration {it+1}/{max_iters}...")
        # sort wolves by fitness (lower better)
        idx = np.argsort(fitness)
        wolves = wolves[idx]
        fitness = fitness[idx]

        alpha, beta, delta = wolves[0], wolves[1], wolves[2]
        print(f"[GWO] Alpha={alpha}, Beta={beta}, Delta={delta}, Best fitness={fitness[0]:.4f}")

        # update positions (simple heuristic in integer space)
        new_wolves = []
        for w in wolves:
            # create candidate by combining alpha/beta/delta with small random step
            r1, r2 = np.random.rand(), np.random.rand()
            A1 = 2 * r1 - 1
            C1 = 2 * r2
            candidate = int(np.round(alpha - A1 * abs(C1 * alpha - w)))

            # move towards beta, delta occasionally
            if np.random.rand() < 0.3:
                candidate = int(np.round((candidate + beta)/2))
            if np.random.rand() < 0.2:
                candidate = int(np.round((candidate + delta)/2))

            # clamp to bounds
            candidate = max(p_min, min(p_max, candidate))
            new_wolves.append(candidate)

        wolves = np.array(new_wolves)
        fitness = np.array([fitness_VARMAX((int(w), 0), train_y, val_y, train_exog, val_exog) for w in wolves])

    # final pick best
    best_idx = int(np.argmin(fitness))
    print(f"[GWO] Optimization finished. Best p={wolves[best_idx]} with fitness={fitness[best_idx]:.4f}")
    return int(wolves[best_idx])


def coot_optimize_q(fixed_p, train_y, val_y, train_exog=None, val_exog=None,
                    q_min=0, q_max=5, pop_size=6, max_iters=10):
    """
    Simple Coot-like optimizer to search q in [q_min, q_max] given fixed p.
    Returns best_q (int).
    """
    # init population
    coots = np.random.randint(q_min, q_max+1, size=pop_size)
    fitness = np.array([fitness_VARMAX((fixed_p, int(c)), train_y, val_y, train_exog, val_exog) for c in coots])

    for it in range(max_iters):
        print(f"[COA] Iteration {it+1}/{max_iters}...")
        idx = np.argsort(fitness)
        coots = coots[idx]
        fitness = fitness[idx]
        best = coots[0]
        print(f"[COA] Best q={best}, fitness={fitness[0]:.4f}")


        # Coot behaviors: random walk, group follow, leader follow, dive
        new_coots = []
        for c in coots:
            if np.random.rand() < 0.4:
                # small random step around best
                candidate = best + np.random.randint(-1, 2)
            elif np.random.rand() < 0.2:
                # move towards group mean
                candidate = int(round(np.mean(coots)))
            else:
                # exploratory jump
                candidate = c + np.random.randint(-2, 3)

            candidate = max(q_min, min(q_max, candidate))
            new_coots.append(candidate)

        coots = np.array(new_coots)
        fitness = np.array([fitness_VARMAX((fixed_p, int(c)), train_y, val_y, train_exog, val_exog) for c in coots])

    best_idx = int(np.argmin(fitness))
    print(f"[COA] Optimization finished. Best q={coots[best_idx]} with fitness={fitness[best_idx]:.4f}")
    return int(coots[best_idx])


def optimize_orders_dual_metaheuristic(train_y_opt, val_y, train_exog_opt=None, val_exog=None,
                                       p_range=(1,5), q_range=(0,5)):
    """
    Use GWO to find p, then COA to find q (given p).
    p_range and q_range are tuples (min, max).
    """
    p_min, p_max = p_range
    q_min, q_max = q_range

    print("Running Grey Wolf Optimizer (GWO) to search AR order p ...")
    best_p = grey_wolf_optimize_p(train_y_opt, val_y, train_exog_opt, val_exog,
                                  p_min=p_min, p_max=p_max, pop_size=8, max_iters=12)
    print(f"GWO found p = {best_p}")

    print("Running Coot Optimization (COA) to search MA order q (conditional on best p) ...")
    best_q = coot_optimize_q(best_p, train_y_opt, val_y, train_exog_opt, val_exog,
                             q_min=q_min, q_max=q_max, pop_size=8, max_iters=12)
    print(f"COA found q = {best_q}")

    return best_p, best_q

# -------------------------
# ---- Main pipeline (modified)
# -------------------------
def run_multivariate_pipeline_meta(df, plot_all=False,
                                   val_fraction=0.20,
                                   p_search_range=(1,5),
                                   q_search_range=(0,5)):
    # 1) Split
    train, test = make_splits(df)

    # 2) Build Y (targets) and exog (features)
    train_y = train[TARGET_COLS].copy()
    test_y  = test[TARGET_COLS].copy()

    train_exog_raw, exog_cols = build_exog(train)
    test_exog_raw  = test[exog_cols].copy()

    # 3) Clean exog and align
    train_exog, test_exog = clean_exog(train_exog_raw, test_exog_raw)

    # 4) Optionally difference targets for stationarity (here we skip differencing to keep original scale)
    dy_train = train_y
    last_train_level = None

    # 5) Align exog rows with train_y index
    train_exog_aligned = train_exog.loc[dy_train.index]

    # ---- Split train into optimization-train and validation for hyperparameter search ----
    val_size = max(1, int(len(dy_train) * val_fraction))
    train_y_opt = dy_train.iloc[:-val_size]
    val_y = dy_train.iloc[-val_size:]

    train_exog_opt = train_exog_aligned.iloc[:-val_size] if train_exog_aligned is not None else None
    val_exog = train_exog_aligned.iloc[-val_size:] if train_exog_aligned is not None else None

    # 6) Optimize (p,q) using dual metaheuristic on the small validation split
    try:
        best_p, best_q = optimize_orders_dual_metaheuristic(
            train_y_opt, val_y,
            train_exog_opt, val_exog,
            p_range=p_search_range, q_range=q_search_range
        )
    except Exception as e:
        print(f"[WARN] Metaheuristic optimization failed: {e}. Falling back to defaults p=1,q=0")
        best_p, best_q = 1, 0

    print(f"Best VARMAX order found by metaheuristics: p={best_p}, q={best_q}")

    # 7) Fit final VARMAX with optimized orders on the entire training set and forecast test
    steps = len(test)
    try:
        final_forecast, fitted_model = fit_varmax_and_forecast(
            train_y=dy_train,
            test_h=steps,
            train_exog=train_exog_aligned,
            test_exog=test_exog,
            order_tuple=(best_p, best_q)
        )
        used_model = f"Metaheuristic-optimized VARMAX (order=({best_p},{best_q}))"
    except Exception as e:
        print(f"[WARN] Final VARMAX fit failed ({e}). Falling back to VAR (no exog).")
        final_forecast, fitted_model = fit_var_fallback(dy_train, steps)
        used_model = "VAR (no exogenous features)"

    # if we had differenced earlier we'd invert here; we skipped differencing so keep as-is
    level_forecast = final_forecast

    # Align forecast index with test index (force)
    level_forecast.index = test_y.index

    # 8) Evaluate against actual test levels
    metrics = evaluate_forecasts(test_y, level_forecast)

    # 9) Join predictions with actuals for convenience
    out = test_y.join(level_forecast.add_suffix('_pred'), how='left')

    print(f"\nModel used: {used_model}")
    print("\nForecast metrics (test period):")
    print(metrics.round(4))

    # plot residuals if possible
    try:
        residuals = fitted_model.resid
        residuals.plot(subplots=True, figsize=(12,8))
    except Exception:
        pass

    # 10) Optional plotting of series
    if plot_all:
        for series in TARGET_COLS:
            plt.figure(figsize=(8,4))
            plt.plot(train.index, train[series], label=f"{series} (Train)")
            plt.plot(test.index, test[series], label=f"{series} (Test)", color="blue")
            plt.plot(out.index, out[f"{series}_pred"], label=f"{series} (Forecast)", color="orange")
            plt.title(f"{series}: Actual vs Forecast")
            plt.legend()
            plt.tight_layout()
            plt.show()
    
    # Export fitted model as pickle
    try:
        with open("VARMAX_final_model.pkl", "wb") as f:
            pickle.dump(fitted_model, f)
        print("✅ Model exported to VARMAX_final_model.pkl")
    except Exception as e:
        print(f"[WARN] Could not export model: {e}")


    return {
        'predictions_vs_actuals': out,
        'metrics': metrics,
        'model_summary': str(fitted_model.summary()) if hasattr(fitted_model, 'summary') else None,
        'best_order': (best_p, best_q)
    }

# -------------------------
# Example usage (uncomment and point to your df)
# -------------------------
# df = pd.read_csv("../data/feature-engineered/recession_probability.csv")
# result = run_multivariate_pipeline_meta(df, plot_all=True)
# print(result['best_order'])
# print(result['metrics'])


In [12]:
results = run_multivariate_pipeline_meta(dim_reduced_df)
preds = results['predictions_vs_actuals']   # contains actuals + *_pred columns for all 12 series
metrics = results['metrics']                # MAE/RMSE/MAPE per series
print(results['model_summary'])             # Optional: model summary

for col in preds.columns:
    if not col.endswith("_pred"):  # Only pick actual series
        series = col
        if f"{series}_pred" in preds.columns:  # Ensure prediction exists
            preds[[series, f"{series}_pred"]].plot(
                title=f"{series}: Actual vs Forecast (Test)",
                figsize=(8, 5)
            )
            plt.xlabel("Time")
            plt.ylabel(series)
            plt.show()

Train: 1967-02-01 → 2019-12-01 (635)
Test : 2020-01-01 → 2025-05-01 (65)
Running Grey Wolf Optimizer (GWO) to search AR order p ...
[GWO] Iteration 1/12...
[GWO] Alpha=4, Beta=4, Delta=3, Best fitness=3064.0225
[GWO] Iteration 2/12...
[GWO] Alpha=5, Beta=5, Delta=4, Best fitness=3054.7552
[GWO] Iteration 3/12...
[GWO] Alpha=5, Beta=5, Delta=5, Best fitness=3054.7552
